In [3]:
import os
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
data = pd.read_csv("shl_catalog_listing.csv")
data.columns

Index(['Assessment Name', 'Assessment URL', 'Remote Testing Support',
       'Adaptive/IRT Support', 'Test Type', 'Duration'],
      dtype='object')

In [7]:
with pd.option_context('display.max_colwidth', None):
    display(data['Assessment URL'].head())

0                      https://www.shl.com/solutions/products/product-catalog/view/account-manager-solution/
1        https://www.shl.com/solutions/products/product-catalog/view/administrative-professional-short-form/
2                       https://www.shl.com/solutions/products/product-catalog/view/agency-manager-solution/
3    https://www.shl.com/solutions/products/product-catalog/view/apprentice-8-0-job-focused-assessment-4261/
4         https://www.shl.com/solutions/products/product-catalog/view/apprentice-8-0-job-focused-assessment/
Name: Assessment URL, dtype: object

In [ ]:
import re
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

def fetch_duration(url):
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    
    duration = soup.find_all("div", class_="product-catalogue-training-calendar__row typ")
    duration = duration[3].text.strip()
    if "Approximate Completion Time in minutes = " in duration:
        duration = duration.split("Approximate Completion Time in minutes = ")[1].strip()
        regex = re.search(r'\d+', duration)
    return regex.group()
url = 'https://www.shl.com/solutions/products/product-catalog/view/account-manager-solution/'
print(fetch_duration(url))

49


In [58]:
data['Duration'] = data['Assessment URL'].apply(fetch_duration)
data['Duration'] = data['Duration'].astype(int)

In [59]:
data.head()

,Assessment Name,Assessment URL,Remote Testing Support,Adaptive/IRT Support,Test Type,Duration
0,Account Manager Solution,https://www.shl.com/solutions/products/product...,Yes,Yes,"C, P, A, B",49
1,Administrative Professional - Short Form,https://www.shl.com/solutions/products/product...,Yes,Yes,"A, K, P",36
2,Agency Manager Solution,https://www.shl.com/solutions/products/product...,Yes,Yes,"A, B, P, S",51
3,Apprentice + 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,Yes,NaN,"B, P",30
4,Apprentice 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,Yes,NaN,"B, P",20


In [60]:
data.to_csv("shl_catalog_listing.csv", index=False)